# 第7章 高级的深度学习最佳实践

本章包括以下内容:
- Keras 函数式 API
- 使用 Keras 回调函数
- 使用 TensorBoard 可视化工具
- 开发最先进模型的重要最佳实践

## 7.1 不用 Sequential 模型的解决方案:Keras 函数式 API

### 代码清单 7-1 用函数式 API 实现双输入问答模型

In [10]:
from keras.models import Model
from keras import layers
from keras import Input


text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_7 (LS

### 代码清单 7-2 将数据输入到多输入模型中

In [11]:
import numpy as np
import keras


num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

model.fit([text, question], answers, epochs=10, batch_size=128)
# 使用输入组成的字典来拟合 (只有对输入进行命名之后才能用这种方法)
# model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2145 - acc: 0.0030
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1988 - acc: 0.0300
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1572 - acc: 0.0040
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0766 - acc: 0.0060
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0109 - acc: 0.0060
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9314 - acc: 0.0060
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8410 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7584 - acc: 0.0150
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6830 - acc: 0.0190
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6238 - acc: 0.0180


### 代码清单 7-3 用函数式 API 实现一个三输出模型

In [1]:
from keras import layers
from keras import Input
from keras.models import Model


vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalAvgPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.summary()

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

### 代码清单 7-4 多输出模型的编译选项:多重损失

```python
model.compile(optimizer='rmsprop',
              loss=['mae', 'categorical_crossentropy', 'binary_crossentropy'])

# 与上述写法等效(只有输出层具有名称时才能采用这种写法)
model.compile(optimizer='rmsprop',
      loss={'age': 'mae', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'})
```

### 代码清单 7-5 多输出模型的编译选项:损失加权

```python
# 严重不平衡的损失贡献会导致模型表示针对单个损失值最大的任务优先进行优化
# 比如,用于年龄回归任务的均方误差(MSE)损失值通常在 3~5 左右,而用于性别分类任务的交叉熵损失值可能低至 0.1。
# 在这种情况下,为了平衡不同损失的贡献,我们可以让交叉熵损失的权重取 10,而 MSE 损失的权重取 0.5

model.compile(optimizer='rmsprop',
              loss=['mae', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1.0, 10.])

# 与上述写法等效(只有输出层具有名称时才能采用这种写法)
model.compile(optimizer='rmsprop',
              loss={'age': 'mae',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})
```

### 代码清单 7-6 将数据输入到多输出模型中

```python
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# 与上述写法等效(只有输出层具有名称时才能采用这种写法)
model.fit(posts,
          {'age': age_targets,
           'income': income_targets,
           'gender': gender_targets},
          epochs=10, batch_size=64)
```

## 7.2 使用 Keras 回调函数和 TensorBoard 来检查并监控深度学习模型

### 代码清单 7-7 使用了 TensorBoard 的文本分类模型

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence


max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
# model.add(layers.Conv1D(32, 7, activation='relu'))
# model.add(layers.MaxPooling1D(5))
# model.add(layers.Conv1D(32, 7, activation='relu'))
# model.add(layers.GlobalMaxPooling1D())
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 64000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2048032   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,304,065
Trainable params: 2,304,065
Non-trainable params: 0
_________________________________________________________________


### 代码清单 7-9 使用一个 TensorBoard 回调函数来训练模型

In [2]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
    )
]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 2s 110us/step - loss: 0.5944 - acc: 0.6490 - val_loss: 0.3578 - val_acc: 0.8490
Epoch 2/20
20000/20000 [==============================] - 2s 100us/step - loss: 0.2574 - acc: 0.8963 - val_loss: 0.3264 - val_acc: 0.8618
Epoch 3/20
20000/20000 [==============================] - 2s 98us/step - loss: 0.1149 - acc: 0.9604 - val_loss: 0.3979 - val_acc: 0.8562
Epoch 4/20
20000/20000 [==============================] - 2s 99us/step - loss: 0.0343 - acc: 0.9920 - val_loss: 0.6450 - val_acc: 0.8218
Epoch 5/20
20000/20000 [==============================] - 2s 100us/step - loss: 0.0080 - acc: 0.9986 - val_loss: 0.6467 - val_acc: 0.8494
Epoch 6/20
20000/20000 [==============================] - 2s 99us/step - loss: 0.0027 - acc: 0.9995 - val_loss: 0.7552 - val_acc: 0.8424
Epoch 7/20
20000/20000 [==============================] - 2s 98us/step - loss: 5.9579e-04 - acc: 0.9999 - val_loss: 0.8677 - v